In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# Trying different layouts for TF models.
# This was inspired by http://web.stanford.edu/class/cs20si/lectures/notes_04.pdf
# Split between assembly and training and reduce verbosity.

In [3]:
class MLP(object):
    """ Multi layer perceptron"""
    # https://github.com/nlintz/TensorFlow-Tutorials/blob/master/04_modern_net.py
    
    def __init__(self, sess, n_in, n_hidden, n_out, lr):
        self.sess = sess
        self.n_in = n_in
        self.n_hidden = n_hidden
        self.n_out = n_out
        self.lr = lr
        self.assemble()
        
    def model(self, X, w_h, w_h2, w_o, p_keep_in, p_keep_h):
        X = tf.nn.dropout(X, p_keep_in)
        h1 = tf.nn.relu(tf.matmul(X, w_h))
        h1 = tf.nn.dropout(h1, p_keep_h)
        h2 = tf.nn.relu(tf.matmul(h1, w_h2))
        h2 = tf.nn.dropout(h2, p_keep_h)
        return tf.matmul(h2, w_o)
    
    def init_weights(self, shape):
        return tf.Variable(tf.random_normal(shape, stddev=0.01))
        
    def assemble(self):
        """ Builds graph """
        # Create placeholders.
        self.X = tf.placeholder("float", [None, self.n_in])
        self.Y = tf.placeholder("float", [None, self.n_out])
        self.p_keep_in = tf.placeholder("float")
        self.p_keep_h = tf.placeholder("float")
      
        # Create variables and initialize.
        self.w_h1 = self.init_weights([self.n_in, self.n_hidden])
        self.w_h2 = self.init_weights([self.n_hidden, self.n_hidden])
        self.w_o = self.init_weights([self.n_hidden, self.n_out])
  
        # Creates inference model.
        self.logits = self.model(self.X, self.w_h1, self.w_h2, self.w_o, self.p_keep_in, self.p_keep_h)
        
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
        self.train_op = tf.train.RMSPropOptimizer(self.lr, 0.9).minimize(self.loss)
        self.predict_op = tf.argmax(self.logits, axis=1)
    
    def train(self, trX, trY, teX, teY):
        """ Trains data on graph """
        tf.global_variables_initializer().run()
        for i in range(10):
            for start, end in zip(range(0, len(trX), 128), range(128, len(trX)+1, 128)):
                self.sess.run(self.train_op, feed_dict={self.X: trX[start:end], self.Y: trY[start:end],
                                          self.p_keep_in: 0.8, self.p_keep_h: 0.5})
            print(i, np.mean(np.argmax(teY, axis=1) ==
                         self.sess.run(self.predict_op, feed_dict={self.X: teX, 
                                                         self.p_keep_in: 1.0,
                                                         self.p_keep_h: 1.0})))
            

            
# Set up data   
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

# Set up graph and train
with tf.Session() as sess:
    model = MLP(sess, 784, 625, 10, lr=0.001)
    model.train(trX, trY, teX, teY)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
(0, 0.93679999999999997)
(1, 0.95950000000000002)
(2, 0.97160000000000002)
(3, 0.97509999999999997)
(4, 0.97550000000000003)
(5, 0.97670000000000001)
(6, 0.97619999999999996)
(7, 0.97919999999999996)
(8, 0.97919999999999996)
(9, 0.97989999999999999)
